In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end
 
#Pkg.build("PyCall")

In [ ]:
using JuliaProbo

In [ ]:
xlim = [-5.0, 5.0]
ylim = [-5.0, 5.0]
world = PuddleWorld(xlim, ylim)
push!(world, Puddle([-2.0, 0.0], [0.0, 2.0], 0.1))
push!(world, Puddle([-0.5, -2.0], [2.5, 1.0], 0.1))

In [ ]:
import Seaborn, Images, PyPlot
import Plots: @animate, gif

sampling_num = 10
dp = DynamicProgramming([0.1, 0.1, pi/20], Goal(-3.0, -3.0), dt=0.1);
init_value(dp)
init_policy(dp)
init_state_transition_probs(dp, 0.1, sampling_num)
init_depth(dp, world, sampling_num)

In [ ]:
action_switch_rate = 1.0
sweep_num = 0
while action_switch_rate > 0.001 && sweep_num < 1000
    action_switch_rate = policy_iteration_sweep(dp)
    sweep_num += 1
    if (sweep_num-1) % 5 == 0
        println("$(sweep_num)th: $(action_switch_rate)")
    end
end
println("$(sweep_num)th: $(action_switch_rate)")

In [ ]:
p = zeros(Float64, dp.index_nums...)
for index in dp.indices
    p[index...] = sum(dp.policy_[index..., :])
end

Seaborn.heatmap(rotl90(p[:, :, 20]))

In [ ]:
PyPlot.clf()
Seaborn.heatmap(rotl90(dp.value_function_[:, :, 20]))

In [ ]:
import Seaborn, PyPlot
import Plots: @animate, gif
using Images

sampling_num = 10
dp = DynamicProgramming([0.1, 0.1, pi/20], Goal(-3.0, -3.0), dt=0.1);
init_value(dp)
init_policy(dp)
init_state_transition_probs(dp, 0.1, sampling_num)
init_depth(dp, world, sampling_num)
file_names = Vector{String}(undef, 0)

action_switch_rate = 1.0
sweep_num = 0
image_num = 0
while action_switch_rate > 0.0001 && sweep_num < 1000
    if (sweep_num-1) % 5 == 0
        Seaborn.heatmap(rotl90(dp.value_function_[:, :, 20]))
        fname = "ch10_dp2_policy_iteration_tmp$(sweep_num).png"
        image_num += 1
        Seaborn.savefig(fname)
        PyPlot.clf()
        push!(file_names, fname)
    end
    action_switch_rate = policy_iteration_sweep(dp)
    sweep_num += 1
    if (sweep_num-1) % 5 == 0
        println("$(sweep_num)th: $(action_switch_rate)")
    end
end
anim = @animate for i in 1:image_num
    Plots.plot(Images.load(file_names[i]), axis=nothing)
    rm(file_names[i], force=true)
end
gif(anim, "images/ch10_dp2_policy_iteration.gif", fps=5)